<a href="https://colab.research.google.com/github/TirendazAcademy/Cancer-Detection-Project/blob/main/Cancer_Gradio_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio -q
!pip install comet_ml -q

In [ ]:
import comet_ml
import torch
from torch import nn
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
import gradio as gr

In [ ]:
title = "Cancer Detection"
description = "Image classification with histopathologic images"
article = "<p style='text-align: center'><a href='https://github.com/TirendazAcademy'>Github Repo</a></p>"

In [ ]:
# The model architecture
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrain_model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.pretrain_model.eval()
        for param in self.pretrain_model.parameters():
            param.requires_grad = False       
        self.pretrain_model.fc = nn.Sequential(
            nn.Linear(self.pretrain_model.fc.in_features, 1024),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(1024,2)
        )
    def forward(self, input):
        output=self.pretrain_model(input)
        return output

In [ ]:
from comet_ml import API
api = API(api_key= "your-api-key")

In [ ]:
# Download a Registry Model:
api.download_registry_model("tirendaz", "my-pl-model", "1.0.0", output_path="./", expand=True)

COMET INFO: Downloading registry model 'my-pl-model', version '1.0.0', stage None from workspace 'tirendaz'...
COMET INFO: Unzipping model to '/content' ...
COMET INFO: done!


In [ ]:
# Loading the model
model = ImageClassifier()
model.load_state_dict(torch.load('/content/model-data/comet-torch-model.pth'))

<All keys matched successfully>

In [ ]:
def predict(inp):
    image_transform = transforms.Compose([ transforms.Resize(size=(224,224)), transforms.ToTensor()])
    labels = ['normal', 'cancer']
    inp = image_transform(inp).unsqueeze(dim=0)
    with torch.no_grad():
        prediction = torch.nn.functional.softmax(model(inp))
        confidences = {labels[i]: float(prediction.squeeze()[i]) for i in range(len(labels))}    
    return confidences

In [ ]:
gr.Interface(fn=predict, 
             inputs=gr.Image(type="pil"),
             outputs=gr.Label(num_top_classes=2),
             title=title,
             description=description,
             article=article,
             examples=['image-1.jpg', 'image-2.jpg']).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>